# 🦺 Smart Safety Vision - YOLOv8 Model Training

This notebook guides you through training a custom YOLOv8 model for PPE detection.

## 📋 Prerequisites
- Google Colab (recommended) or local GPU
- PPE detection dataset
- Ultralytics YOLOv8 installed

## 1️⃣ Setup Environment

In [ ]:
# Install required packages
!pip install ultralytics roboflow opencv-python matplotlib

# Import libraries
from ultralytics import YOLO
import os
from pathlib import Path
import yaml
import matplotlib.pyplot as plt
from IPython.display import Image, display

print("✅ Environment setup complete!")

## 2️⃣ Download Dataset

### Option A: Using Roboflow

In [ ]:
# Install Roboflow
!pip install roboflow

from roboflow import Roboflow

# Initialize Roboflow (get API key from roboflow.com)
rf = Roboflow(api_key="YOUR_API_KEY_HERE")

# Download PPE detection dataset
project = rf.workspace("your-workspace").project("ppe-detection")
dataset = project.version(1).download("yolov8")

print(f"✅ Dataset downloaded to: {dataset.location}")

### Option B: Using Custom Dataset

In [ ]:
# If you have a custom dataset, organize it in YOLO format:
# dataset/
#   ├── train/
#   │   ├── images/
#   │   └── labels/
#   ├── valid/
#   │   ├── images/
#   │   └── labels/
#   └── data.yaml

# Upload your dataset to Colab or specify path
dataset_path = "path/to/your/dataset"
print(f"Using dataset at: {dataset_path}")

## 3️⃣ Create Data Configuration

In [ ]:
# Create data.yaml configuration file
data_config = {
    'path': dataset.location,  # or your dataset_path
    'train': 'train/images',
    'val': 'valid/images',
    'test': 'test/images',  # optional
    'nc': 5,  # number of classes
    'names': ['helmet', 'no_helmet', 'vest', 'no_vest', 'person']
}

# Save configuration
with open('data.yaml', 'w') as f:
    yaml.dump(data_config, f)

print("✅ Data configuration created!")
print(yaml.dump(data_config, default_flow_style=False))

## 4️⃣ Load Pretrained Model

In [ ]:
# Load YOLOv8 nano model (fastest, good for real-time)
model = YOLO('yolov8n.pt')

# Alternative models:
# model = YOLO('yolov8s.pt')  # Small - better accuracy
# model = YOLO('yolov8m.pt')  # Medium - even better accuracy
# model = YOLO('yolov8l.pt')  # Large - best accuracy, slower

print("✅ Model loaded successfully!")

## 5️⃣ Train Model

In [ ]:
# Training parameters
results = model.train(
    data='data.yaml',
    epochs=100,              # Number of training epochs
    imgsz=640,              # Image size
    batch=16,               # Batch size (adjust based on GPU memory)
    name='ppe_detection',   # Experiment name
    patience=20,            # Early stopping patience
    save=True,              # Save checkpoints
    device=0,               # GPU device (0 for first GPU, 'cpu' for CPU)
    workers=8,              # Number of workers
    project='runs/detect',  # Project directory
    exist_ok=True,          # Overwrite existing project
    pretrained=True,        # Use pretrained weights
    optimizer='auto',       # Optimizer (auto, SGD, Adam, AdamW)
    verbose=True,           # Verbose output
    seed=42,                # Random seed
    deterministic=True,     # Deterministic training
    single_cls=False,       # Train as single-class
    rect=False,             # Rectangular training
    cos_lr=False,           # Cosine learning rate scheduler
    close_mosaic=10,        # Disable mosaic augmentation for final epochs
    resume=False,           # Resume training
    amp=True,               # Automatic Mixed Precision
    fraction=1.0,           # Dataset fraction to train on
    profile=False,          # Profile ONNX and TensorRT speeds
    # Data augmentation
    hsv_h=0.015,           # HSV-Hue augmentation
    hsv_s=0.7,             # HSV-Saturation augmentation
    hsv_v=0.4,             # HSV-Value augmentation
    degrees=0.0,           # Rotation augmentation
    translate=0.1,         # Translation augmentation
    scale=0.5,             # Scale augmentation
    shear=0.0,             # Shear augmentation
    perspective=0.0,       # Perspective augmentation
    flipud=0.0,            # Vertical flip augmentation
    fliplr=0.5,            # Horizontal flip augmentation
    mosaic=1.0,            # Mosaic augmentation
    mixup=0.0,             # Mixup augmentation
    copy_paste=0.0         # Copy-paste augmentation
)

print("\n✅ Training complete!")

## 6️⃣ Evaluate Model

In [ ]:
# Validate the model
metrics = model.val()

# Print metrics
print("\n📊 Model Performance Metrics:")
print(f"mAP50: {metrics.box.map50:.4f}")
print(f"mAP50-95: {metrics.box.map:.4f}")
print(f"Precision: {metrics.box.mp:.4f}")
print(f"Recall: {metrics.box.mr:.4f}")

## 7️⃣ Visualize Training Results

In [ ]:
# Display training curves
results_dir = 'runs/detect/ppe_detection'

# Show results
display(Image(filename=f'{results_dir}/results.png', width=800))

# Show confusion matrix
display(Image(filename=f'{results_dir}/confusion_matrix.png', width=600))

# Show validation batch predictions
display(Image(filename=f'{results_dir}/val_batch0_pred.jpg', width=800))

## 8️⃣ Test Model on Sample Images

In [ ]:
# Load best model
best_model = YOLO(f'{results_dir}/weights/best.pt')

# Test on sample images
test_images = ['path/to/test/image1.jpg', 'path/to/test/image2.jpg']

for img_path in test_images:
    results = best_model(img_path)
    
    # Display results
    for r in results:
        im_array = r.plot()  # Plot results
        plt.figure(figsize=(12, 8))
        plt.imshow(im_array[..., ::-1])  # Convert BGR to RGB
        plt.axis('off')
        plt.title(f'Detection Results: {img_path}')
        plt.show()

## 9️⃣ Export Model

In [ ]:
# Export to different formats

# 1. ONNX (for faster inference)
best_model.export(format='onnx')
print("✅ Model exported to ONNX")

# 2. TensorRT (for NVIDIA GPUs)
# best_model.export(format='engine')

# 3. TensorFlow Lite (for mobile)
# best_model.export(format='tflite')

# 4. CoreML (for iOS)
# best_model.export(format='coreml')

## 🔟 Download Trained Model

In [ ]:
# If using Google Colab, download the model
from google.colab import files

# Download best weights
files.download(f'{results_dir}/weights/best.pt')

print("✅ Model downloaded! Upload to your project's models/ folder")

## 📊 Performance Optimization Tips

### If accuracy is low:
1. **Increase epochs**: Train for more epochs (150-200)
2. **Use larger model**: Try YOLOv8s or YOLOv8m
3. **More data**: Collect more training images
4. **Data augmentation**: Increase augmentation parameters
5. **Better annotations**: Review and improve label quality

### If inference is slow:
1. **Use smaller model**: YOLOv8n is fastest
2. **Export to ONNX**: Faster inference
3. **Reduce image size**: Use 416 or 320 instead of 640
4. **Use GPU**: Enable CUDA if available
5. **Batch processing**: Process multiple frames together

### If memory issues:
1. **Reduce batch size**: Try 8 or 4
2. **Smaller image size**: Use 416 instead of 640
3. **Use gradient accumulation**: Simulate larger batches
4. **Close other applications**: Free up memory

## 🎯 Next Steps

1. Download the `best.pt` file
2. Place it in your project's `models/` folder
3. Update `config.yaml` with the model path
4. Run the main detection system: `python main.py`
5. Launch dashboard: `streamlit run dashboard/app.py`

Happy detecting! 🦺